In [ ]:
# Use if you run the notebook on Google colab
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
!pip install mglearn

# 4: Classification Metrics

## Imports

In [ ]:
import os
import sys

sys.path.append("/content/drive/MyDrive/50603/code")
os.chdir('/content/drive/MyDrive/50603')

import IPython
import matplotlib.pyplot as plt
import mglearn
import numpy as np
import pandas as pd
from IPython.display import Image, HTML, display
from plotting_functions import *
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from utils import *

%matplotlib inline
pd.set_option("display.max_colwidth", 200)

In [ ]:
# Changing global matplotlib settings for confusion matrix.
plt.rcParams["xtick.labelsize"] = 18
plt.rcParams["ytick.labelsize"] = 18

## Learning outcomes

From this lecture, students are expected to be able to:

- Explain why accuracy is not always the best metric in ML.
- Explain components of a confusion matrix.
- Define precision, recall, and f1-score and use them to evaluate different classifiers.
- Interpret and use ROC curves and ROC AUC using `scikit-learn`.  

<br><br><br><br>

## Evaluation metrics for binary classification: Motivation

### Dataset for demonstration

- Let's classify fraudulent and non-fraudulent transactions using Kaggle's [Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud) data set.

In [ ]:
cc_df = pd.read_csv("data/creditcard.csv", encoding="latin-1")
train_df, test_df = train_test_split(cc_df, test_size=0.3, random_state=111)
train_df.head()

In [ ]:
train_df.shape

- Good size dataset
- For confidentially reasons, it only provides transformed features with PCA, which is a popular dimensionality reduction technique.

### Exploratory Data Analysis (EDA)

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

In [ ]:
# no missing data as all columns have same count:
train_df.describe().loc["count"].unique()

- We do not have categorical features. All features are numeric.
- We have to be careful about the `Time` and `Amount` features.
- We could scale `Amount`.
- Do we want to scale time?
    - In this lecture we'll do it's probably not the best thing to do.
    - We'll learn about time series briefly later in the course.

Let's separate `X` and `y` for train and test splits.

In [ ]:
X_train_big, y_train_big = train_df.drop(columns=["Class"]), train_df["Class"]
X_test, y_test = test_df.drop(columns=["Class"]), test_df["Class"]

- It's **easier to demonstrate** evaluation metrics using an explicit **validation set instead of using cross-validation**.
- So let's create a validation set.
- Our data is large enough so it shouldn't be a problem.


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_big, y_train_big, test_size=0.3, random_state=123
)

### Classifier

Let's try `LogisticRegression`. Don't worry about the logistic regression for now. It is a simple and widely used classification model.

In [ ]:
pipe_lr = make_pipeline(StandardScaler(), LogisticRegression())
pd.DataFrame(cross_validate(pipe_lr, X_train, y_train, return_train_score=True)).mean()

- `.score` by default returns accuracy which is
$$accuracy = \frac{correct\ predictions}{total\ examples}$$
- Is accuracy a good metric here?
- Is there anything more informative than accuracy that we can use here?

Let's dig a little deeper.

<br><br><br><br>

## Confusion matrix

One way to get a better understanding of the errors is by looking at
- false positives (type I errors), where the model incorrectly spots examples as fraud
- false negatives (type II errors), where it's missing to spot fraud examples

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

pipe_lr.fit(X_train, y_train)
disp = ConfusionMatrixDisplay.from_estimator(
    pipe_lr,
    X_valid,
    y_valid,
    display_labels=["Non fraud", "fraud"],
    values_format="d",
    cmap=plt.cm.Blues,
    colorbar=False,
)

In [ ]:
from sklearn.metrics import confusion_matrix

predictions = pipe_lr.predict(X_valid)
TN, FP, FN, TP = confusion_matrix(y_valid, predictions).ravel()
plot_confusion_matrix_example(TN, FP, FN, TP)

- **Perfect** prediction has all values **down the diagonal**
- **Off diagonal** entries can often tell us about what is being **mis-predicted**

### What is "positive" and "negative"?

- Two kinds of binary classification problems
    - Distinguishing between two classes
    - **Spotting** a class (spot fraud transaction, spot spam, spot disease)
- In case of spotting problems, the thing that we are interested in spotting is considered "**positive**".
- Above we wanted to **spot fraudulent** transactions and so they are "positive".

You can get a numpy array of confusion matrix, and you can *unpack* it into its elements using numpy `ravel()` as follows:

In [ ]:
from sklearn.metrics import confusion_matrix

predictions = pipe_lr.predict(X_valid)  # note that pipe_lr must have already been fitted using train data
cm = confusion_matrix(y_valid, predictions)

In [ ]:
print("Confusion matrix for fraud dataset:")
cm

In [ ]:
cm.ravel()  # get a flattened array

In [ ]:
TN, FP, FN, TP = cm.ravel()  # unpack cm elements
TN, FP, FN, TP

### Confusion matrix with cross-validation

- You can also calculate confusion matrix with cross-validation using the `cross_val_predict` method.
- Then you need to use `ConfusionMatrixDisplay`'s **`from_predictions`** to draw confusion matrix.

In [ ]:
from sklearn.model_selection import cross_val_predict

confusion_matrix(y_train, cross_val_predict(pipe_lr, X_train, y_train))

In [ ]:
ConfusionMatrixDisplay.from_predictions(
    y_train,
    cross_val_predict(pipe_lr, X_train, y_train),
    display_labels=["Non fraud", "fraud"],
    values_format="d",
    cmap=plt.cm.Blues,
);

<br><br><br><br>

## Precision, recall, f1 score

- We have been using `.score` to assess our models, which returns **accuracy by default**.
- Accuracy is misleading when we have class imbalance.
- We need other metrics to assess our models.

- We'll discuss three commonly used metrics which are **based on confusion matrix**:
    - *recall*
    - *precision*
    - *f1 score*
- Note that these metrics will only help us **assessing our model**.
- Later we'll talk about a few ways to address class imbalance problem.

In [ ]:
from sklearn.metrics import confusion_matrix

pipe_lr = make_pipeline(StandardScaler(), LogisticRegression())
pipe_lr.fit(X_train, y_train)
predictions = pipe_lr.predict(X_valid)
cm = confusion_matrix(y_valid, predictions)
TN, FP, FN, TP = cm.ravel()
print("TN, FP, FN, TP:", TN, FP, FN, TP, '\n')
cm

### Recall

Among all positive examples, how many did you identify?
$$ recall = \frac{TP}{TP+FN} = \frac{TP}{\#positives} $$

In [ ]:
ConfusionMatrixDisplay.from_estimator(
    pipe_lr,
    X_valid,
    y_valid,
    display_labels=["Non fraud", "fraud"],
    values_format="d",
    cmap=plt.cm.Blues,
);

In [ ]:
print("TP = %0.4f, FN = %0.4f" % (TP, FN))
recall = TP / (TP + FN)
print("Recall: %0.4f" % (recall))

### Precision

Among the positive examples you identified, how many were actually positive?

$$ precision = \frac{TP}{TP+FP}$$

In [ ]:
ConfusionMatrixDisplay.from_estimator(
    pipe_lr,
    X_valid,
    y_valid,
    display_labels=["Non fraud", "fraud"],
    values_format="d",
    cmap=plt.cm.Blues,
);

In [ ]:
print("TP = %0.4f, FP = %0.4f" % (TP, FP))
precision = TP / (TP + FP)
print("Precision: %0.4f" % (precision))

### F1-score

- F1-score **combines precision and recall** to give one score, which could be used in hyperparameter optimization, for instance.
- F1-score is a harmonic mean of precision and recall.


$$ f1 = 2 \times \frac{ precision \times recall}{precision + recall}$$


In [ ]:
print("precision: %0.4f" % (precision))
print("recall: %0.4f" % (recall))
f1_score = (2 * precision * recall) / (precision + recall)
print("f1: %0.4f" % (f1_score))

Let's look at all metrics at once on our dataset.

In [ ]:
## Calculate evaluation metrics by ourselves
data = {
    "calculation": [],
    "accuracy": [],
    "error": [],
    "precision": [],
    "recall": [],
    "f1 score": [],
}
data["calculation"].append("manual")
data["accuracy"].append((TP + TN) / (TN + FP + FN + TP))
data["error"].append((FP + FN) / (TN + FP + FN + TP))
data["precision"].append(precision)  # TP / (TP + FP)
data["recall"].append(recall)  # TP / (TP + FN)
data["f1 score"].append(f1_score)  # (2 * precision * recall) / (precision + recall)
df = pd.DataFrame(data)
df

- `scikit-learn` has functions for [these metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics).

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

data["accuracy"].append(accuracy_score(y_valid, pipe_lr.predict(X_valid)))
data["error"].append(1 - accuracy_score(y_valid, pipe_lr.predict(X_valid)))
data["precision"].append(
    precision_score(y_valid, pipe_lr.predict(X_valid), zero_division=1)
)
data["recall"].append(recall_score(y_valid, pipe_lr.predict(X_valid)))
data["f1 score"].append(f1_score(y_valid, pipe_lr.predict(X_valid)))
data["calculation"].append("sklearn")
df = pd.DataFrame(data)
df.set_index(["calculation"])

The scores match.

### Classification report

- There is a convenient function called `classification_report` in `sklearn` which gives this info.

In [ ]:
pipe_lr.classes_

In [ ]:
from sklearn.metrics import classification_report

print("y_valid, not fraud:", len(y_valid) - y_valid.sum())
print("y_valid, fraud:      ", y_valid.sum())
print("X_valid, total:    ", X_valid.shape[0], "\n\n")

print(classification_report(
        y_valid, pipe_lr.predict(X_valid), target_names=["non-fraud", "fraud"], digits=4))

In [ ]:
cr_dict = classification_report(
    y_valid, pipe_lr.predict(X_valid), target_names=["non-fraud", "fraud"], output_dict=True)

cr = pd.DataFrame(cr_dict).T
cr

### Macro average

- You give **equal importance** to all classes and average over all classes.
- In the example above, recall for non-fraud is ~ 1.0 and fraud is 0.63, and so macro average is 0.81.
- See our example calculation below for `macro avg` for `precision`, `recall`, and `f1-score`
- More relevant in case of **multi-class** problems (more later)

### Weighted average

- Weighted by the number of samples in each class.
- Divide by the total number of samples.
- See our example calculation below for `weighted_avg_of_precision`

### Which one to use?
Which one of Weighted or Macro averages is relevant depends upon whether you think:

- each class should have the same weight or
- each sample should have the same weight.

That is, it will be domain/problem dependent.

<br><br>

### Interim summary

- **Accuracy is misleading** when you have class **imbalance**.
- A **confusion matrix** provides a way to **break down errors** made by our model.
- We looked at three metrics based on confusion matrix:
    - precision, recall, f1-score.

- Note that what you consider "positive" (fraud in our case) is important when calculating precision, recall, and f1-score.
- If you flip what is considered positive or negative, we'll end up with different TP, FP, TN, FN, and hence different precision, recall, and f1-scores.

### Evalution metrics overview  
There is a lot of terminology here.

In [ ]:
p = 'img/evaluation-metrics.png'
display(Image(filename=p, width=1000))

<!-- <img src='./img/evaluation-metrics.png' width="1000" height="1000" /> -->

### Cross validation with different metrics

- We can pass different evaluation metrics with `scoring` argument of `cross_validate`.

In [ ]:
scoring = [
    "accuracy",
    "f1",
    "recall",
    "precision",
]  # scoring can be a string, a list, or a dictionary
pipe_lr = make_pipeline(StandardScaler(), LogisticRegression())
scores = cross_validate(
    pipe_lr, X_train_big, y_train_big, return_train_score=True, scoring=scoring
)
pd.DataFrame(scores)

- You can also create [your own scoring function](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html) and pass it to `cross_validate`.

<br><br><br><br>

## Precision-recall curve and ROC curve

- Confusion matrix provides a detailed break down of the errors made by the model.
- But when creating a confusion matrix, we are using "hard" predictions.
- Most classifiers in `scikit-learn` provide `predict_proba` method (or `decision_function`) which provides **degree of certainty** about predictions by the classifier.
- Can we explore the degree of uncertainty to understand and improve the model performance?

Let's revisit the classification report on our fraud detection example.

In [ ]:
pipe_lr = make_pipeline(StandardScaler(), LogisticRegression())
pipe_lr.fit(X_train, y_train);

In [ ]:
y_pred = pipe_lr.predict(X_valid)
print(classification_report(y_valid, y_pred, target_names=["non-fraud", "fraud"], digits=4))

By default, predictions use the **threshold of 0.5**. If `predict_proba` > 0.5, predict "fraud" (positive) else predict "non-fraud" (negative).


In the above code, the function `predict` returns a boolean array, `y_pred`:

In [ ]:
y_pred

In [ ]:
np.unique(y_pred)

We can create the same boolean array `y_pred` directly using `predict_proba` > 0.5:

In [ ]:
# negative class column is 0, and positive class column is 1, so we want [:, 1]
y_pred = pipe_lr.predict_proba(X_valid)[:, 1] > 0.50
print(classification_report(y_valid, y_pred, target_names=["non-fraud", "fraud"], digits=4))

<br>

Now,
- Suppose for your business it is more costly to miss fraudulent transactions and you want to achieve a **recall of at least 75%** for the "fraud" class.
- One way to do this is by **changing the threshold** of `predict_proba`.
    - `predict` returns 1 when `predict_proba`'s probabilities are above 0.5 for the "fraud" class.

**Key idea:**
> **what if we _threshold the probability at a smaller value_ so that we identify more examples as "fraud" examples?**

Let's lower the **threshold to 0.1**. In other words, predict the examples as "fraud" if `predict_proba` > 0.1.  

In [ ]:
y_pred_lower_threshold = pipe_lr.predict_proba(X_valid)[:, 1] > 0.1

In [ ]:
print(classification_report(y_valid, y_pred_lower_threshold, digits=4))

### Operating point

- Now our recall for "fraud" class is >= 0.75.
- Setting a **requirement on a classifier** (e.g., recall of >= 0.75) is called setting the **operating point**.
- It's usually driven by **business goals** and is useful to make performance guarantees to customers.

### Precision/Recall tradeoff

- But there is a trade-off between precision and recall.
- If you identify more things as "fraud",
    - recall is going to increase but
    - there are likely to be more false positives.

Let's sweep through different thresholds.

In [ ]:
def f(threshold):
    preds = pipe_lr.predict_proba(X_valid)[:, 1] > threshold
    print("Threshold: ", np.round(threshold,4))
    print("Precision: ", np.round(precision_score(y_valid, preds),4))
    print("Recall: ", np.round(recall_score(y_valid, preds), 4))
    print("f1 score: ", np.round(f1_score(y_valid, preds), 4))

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive

interactive(
    f,
    threshold=widgets.FloatSlider(min=0, max=0.9, step=0.1, value=0.5),
)

Let's view Precision and Recall for different thresholds as a dataframe.

In [ ]:
thresholds = np.arange(0, 1, 0.1)
thresholds

In [ ]:
pr_dict = {"threshold": [], "precision": [], "recall": [], "f1 score": []}
for threshold in thresholds:
    preds = pipe_lr.predict_proba(X_valid)[:, 1] > threshold
    pr_dict["threshold"].append(threshold)
    pr_dict["precision"].append(precision_score(y_valid, preds))
    pr_dict["recall"].append(recall_score(y_valid, preds))
    pr_dict["f1 score"].append(f1_score(y_valid, preds))

In [ ]:
pr_df = pd.DataFrame(pr_dict).set_index('threshold')
pr_df

In [ ]:
pr_df[['precision', 'recall']].plot();

### Decreasing the threshold

- ***Decreasing the threshold*** means a lower bar for predicting fraud.
    - You are willing to risk more false positives FP⬆ in exchange of more true positives TP⬆.
      - In general, predicted positives (TP + FP)⬆ go up or stay the same
      - In general, predicted negatives (TN + FN)⬇ go down or stay the same
    - recall is likely to go up or stay the same
      - TP⬆ / (TP⬆+FN⬇) so generally recall ⬆
    - precision is likely to go down or stay the same
      - TP⬆ / (TP⬆+FP⬆) so generally precision ⬇


### Increasing the threshold

On the flip side:

- Increasing the threshold means a higher bar for predicting fraud.
    - recall would go down or stay the same but precision is likely to go up
    - occasionally, precision may go down as the denominator for precision is TP+FP.    

### Receiver Operating Characteristic (ROC) curve

- One commonly used tool to analyze the **behavior of classifiers at different thresholds**.
- It considers all possible thresholds for a given classifier given by `predict_proba` and plots false positive rate (FPR) and true positive rate (TPR or recall).
$$ FPR  = \frac{FP}{FP + TN}, TPR = \frac{TP}{TP + FN}$$

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_valid, pipe_lr.predict_proba(X_valid)[:, 1])
plt.plot(fpr, tpr, label="ROC Curve")
plt.xlabel("FPR")
plt.ylabel("TPR (recall)")

default_threshold = np.argmin(np.abs(thresholds - 0.5))

plt.plot(
    fpr[default_threshold],
    tpr[default_threshold],
    "or",
    markersize=10,
    label="threshold 0.5",
)
plt.legend(loc="best");

- The **ideal curve is close to the top left**
    - Ideally, you want a classifier with high recall while keeping low false positive rate.  
- The red dot corresponds to the **threshold of 0.5**, which is used by predict.
- We see that compared to the default threshold, we can achieve a **better recall of around 0.8 without increasing FPR**.

#### Let's compare ROC curve of different classifiers.

In [ ]:
pipe_svc = make_pipeline(StandardScaler(), SVC())

pipe_svc.fit(X_train, y_train)

fpr_lr, tpr_lr, thresholds_lr = roc_curve(
    y_valid, pipe_lr.predict_proba(X_valid)[:, 1])

fpr_svc, tpr_svc, thresholds_svc = roc_curve(
    y_valid, pipe_svc.decision_function(X_valid))

In [ ]:
close_default_lr = np.argmin(np.abs(thresholds_lr - 0.5))
close_zero_svm = np.argmin(np.abs(thresholds_svc))

In [ ]:
plt.plot(fpr_svc, tpr_svc, label="svc")
plt.plot(fpr_lr, tpr_lr, label="logistic regression")
plt.plot(
    fpr_svc[close_zero_svm],
    tpr_svc[close_zero_svm],
    "o",
    markersize=10,
    label="default threshold svc",
    c="b",
)
plt.plot(
    fpr_lr[close_default_lr],
    tpr_lr[close_default_lr],
    "*",
    markersize=10,
    label="default threshold logistic regression",
    c="r",
)

plt.xlabel("False positive rate")
plt.ylabel("True positive rate (Recall)")
plt.legend(loc="best");

### Area under the curve (AUC)

- AUC provides a single meaningful number for the model performance.

In [ ]:
from sklearn.metrics import roc_auc_score

roc_lr = roc_auc_score(y_valid, pipe_lr.predict_proba(X_valid)[:, 1])
roc_svc = roc_auc_score(y_valid, pipe_svc.decision_function(X_valid))
print("AUC for LR: {:.3f}".format(roc_lr))
print("AUC for SVC: {:.3f}".format(roc_svc))


- AUC of **0.5** means **random chance**.
- AUC can be interpreted as evaluating the **ranking** of positive examples.
- What's the probability that a randomly picked positive point has a higher score according to the classifier than a randomly picked point from the negative class.
- AUC of **1.0** means **all positive points have a higher score than all negative points**.

***Important***
> For classification problems with imbalanced classes, using AP score or AUC is often much more meaningful than using accuracy.

Similar to `PrecisionRecallCurveDisplay`, there is a `RocCurveDisplay` function in sklearn.

In [ ]:
from sklearn.metrics import RocCurveDisplay

RocCurveDisplay.from_estimator(pipe_lr, X_valid, y_valid);

### Let's look at all the scores at once

In [ ]:
scoring = ["accuracy", "f1", "recall", "precision", "roc_auc", "average_precision"]
pipe_lr = make_pipeline(StandardScaler(), LogisticRegression())
scores = cross_validate(pipe_lr, X_train_big, y_train_big, scoring=scoring)
pd.DataFrame(scores).mean()

***See Also*** (Recommended)
> Check out [these visualization](https://github.com/dariyasydykova/open_projects/tree/master/ROC_animation) on ROC and AUC.

<br><br><br><br>